In [1]:
import findspark

In [2]:
findspark.init('/home/aditya/spark-3.0.0-bin-hadoop2.7/')

In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [4]:
sc = SparkContext()

In [5]:
ssc = StreamingContext(sc,10)
sqlContext = SQLContext(sc)

In [6]:
sockte_stream = ssc.socketTextStream('127.0.0.1',9990)

In [7]:
line = sockte_stream.window(20)

In [8]:
from collections import namedtuple
fields = ('tag','count')
Tweet = namedtuple('Tweet', fields)

In [9]:
( line.flatMap(lambda text:text.split(" "))
 .filter(lambda word:word.lower.startswith('#'))
 .map(lambda word:(word.lower(),1))
 .reduceByKey(lambda a,b:a+b)
 .map(lambda rec:Tweet(rec[0],rec[1]))
 .foreachRDD(lambda rdd: rdd.toDF().sort(desc('count'))
 .limit(10).registerTempTable('tweets'))
)

## Run tweetCounter.py

In [10]:
ssc.start()

In [12]:
from time import sleep
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [14]:
count = 0
while count <10:
    
    sleep(3)
    top_10_tweets = sqlContext.sql('select tag, count from tweet')
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    sns.plt.figure(figsize =(10,8))
    sns.barplot(x='count',y='tag', data=top_10_df)
    sns.plt.show()
    count = count +1

AnalysisException: Table or view not found: tweet; line 1 pos 23;
'Project ['tag, 'count]
+- 'UnresolvedRelation [tweet]
